# Classification using Pytorch and Torchtext

- bidirectional LSTM on some news
- some labeled data on news....

- Torchtext to help us numericalize and load some data
- Torchtext is backed by PyTorch....so naturally torchtext is quite good
- Torchtext is not meant to replace spacy....spacy is still like better in general
  
- PyTorch to help us make some neural network

In [1]:
import torch, torchtext
from torch import nn

import time

#1. puffer - it's outdated....
#2. spend some money - 300 baht get collab pro

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#reproducibility 
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

cuda


In [2]:
torch.__version__

'1.13.0+cu117'

In [3]:
torchtext.__version__

'0.14.0'

## 1. Load the dataset

Make our life easy by using some ready-to-be-used dataset by torchtext

- in your assignment, i will ask you to use penn treebank.....

In [4]:
#if you are using puffer
# import os
# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'
# from torchtext.datasets import AG_NEWS
# train, test = AG_NEWS()

import csv
data_dp_train = open("train.txt", encoding="utf-8")
data_dp_test = open("test.txt", encoding="utf-8")
train = csv.reader(data_dp_train, skipinitialspace=True)
test = csv.reader(data_dp_test, skipinitialspace=True)

In [5]:
train

## 2. EDA - exploratory data analysis
- check common words
- look at some random sample....how it looks, so that we can design proper neural network
- visualize statistics

In [6]:
next(iter(train))

["(3 (2 (2 The) (2 Rock)) (4 (3 (2 is) (4 (2 destined) (2 (2 (2 (2 (2 to) (2 (2 be) (2 (2 the) (2 (2 21st) (2 (2 (2 Century) (2 's)) (2 (3 new) (2 (2 ``) (2 Conan)))))))) (2 '')) (2 and)) (3 (2 that) (3 (2 he) (3 (2 's) (3 (2 going) (3 (2 to) (4 (3 (2 make) (3 (3 (2 a) (3 splash)) (2 (2 even) (3 greater)))) (2 (2 than) (2 (2 (2 (2 (1 (2 Arnold) (2 Schwarzenegger)) (2 ",
 ')) (2 (2 Jean-Claud) (2 (2 Van) (2 Damme)))) (2 or)) (2 (2 Steven) (2 Segal))))))))))))) (2 .)))']

In [7]:
list(iter(train))[200]

["(1 (2 Frida) (2 (1 (1 (2 is) (2 n't)) (2 (2 (2 that) (2 (2 much) (2 different))) (2 (2 from) (3 (2 many) (2 (2 a) (3 (2 Hollywood) (3 romance))))))) (2 .)))"]

In [8]:
set([y for y, x in list(iter(train))])

set()

In [9]:
train_size = len(list(iter(train)))
train_size

0

In [10]:
train

In [11]:
#i gonna cheat a little bit, not gonna use all...my fans will work too hard....
too_much, train, valid = train.random_split(total_length=train_size, 
                                            weights = {"too_much": 0.7, 
                                                       "smaller_train": 0.2,
                                                       "valid": 0.1},
                                            seed = SEED)

AttributeError: '_csv.reader' object has no attribute 'random_split'

I can't run this term although I try to recheck about installation or search in another source. I'm so sorry.

In [12]:
train_size = len(list(iter(train)))
val_size   = len(list(iter(valid)))
test_size  = len(list(iter(test)))

NameError: name 'valid' is not defined

In [ ]:
train_size, val_size, test_size

## 3. Preprocessing

In [14]:
## 3.1 Tokenizing

from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

#check whether the tokenizer works.....
# tokens    = tokenizer("Chaky likes deep learning very much and wants his \
                    #   student to be number 1 in Asia....")
# tokens

In [15]:
next(iter(train))

StopIteration: 

In [ ]:
## 3.2 Numericalization

from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(data_iter):  #data_iter, e.g., train
    for _, text in data_iter:
        yield tokenizer(text)
        
vocab = build_vocab_from_iterator(yield_tokens(train), specials=['<unk>', '<pad>',
                                                                 '<bos>', '<eos>'])

In [16]:
vocab.set_default_index(vocab["<unk>"]) #if you don't the id of this word, set it unk

NameError: name 'vocab' is not defined

In [17]:
vocab(['Chaky', 'wants', 'his', 'student', 'to', 'be', 'number', '1', '.'])

NameError: name 'vocab' is not defined

In [18]:
id2word = vocab.get_itos()

NameError: name 'vocab' is not defined

In [19]:
id2word[0]

NameError: name 'id2word' is not defined

In [20]:
vocab(['<pad>', '<bos>', '<eos>'])

NameError: name 'vocab' is not defined

In [21]:
len(vocab)

NameError: name 'vocab' is not defined

## 4. FastText embedding

We gonna insert this embedding to the NN on the fly.....

In [ ]:
from torchtext.vocab import FastText
fast_vectors = FastText(language='simple')

In [ ]:
fast_embedding = fast_vectors.get_vecs_by_tokens(vocab.get_itos()).to(device)

In [ ]:
fast_embedding.shape #(vocab size, 300) == (52k, 300)

In [ ]:
#please lookup the fasttext embedding of id 100
fast_embedding[100][:10] #size of 300 dim of this word id 100

## 5. Preparing dataloader

Optional - you can either make your own batch loader....
You can use pytorch dataloader....

In [ ]:
text_pipeline  = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1  #1, 2, 3, 4 ---> 0, 1, 2, 3

In [13]:
'''
why padding????

in the same batch, e.g., batch size = 2

"chaky eat sushi", ==> "chaky", "eat", "sushi" ==> 0, 22, 11, 1, 1
"chaky sleep" ==> "chaky", "sleep" ==> 0, 99, 1, 1, 1

'''

'\nwhy padding????\n\nin the same batch, e.g., batch size = 2\n\n"chaky eat sushi", ==> "chaky", "eat", "sushi" ==> 0, 22, 11, 1, 1\n"chaky sleep" ==> "chaky", "sleep" ==> 0, 99, 1, 1, 1\n\n'

In [22]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence #making each batch same length

pad_ix = vocab['<pad>']

#this function gonna be called by DataLoader
def collate_batch(batch):
    label_list, text_list, length_list = [], [], []
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))  #[3, 1, 0, 2, ]
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64) #[0, 44, 21, 2]
        text_list.append(processed_text)
        length_list.append(processed_text.size(0)) #for padding
        
    return torch.tensor(label_list, dtype=torch.int64), \
        pad_sequence(text_list, padding_value=pad_ix, batch_first=True), \
        torch.tensor(length_list, dtype=torch.int64)

NameError: name 'vocab' is not defined

In [ ]:
batch_size = 64

train_loader = DataLoader(train, batch_size = batch_size,
                          shuffle=True, collate_fn=collate_batch)

val_loader   = DataLoader(valid, batch_size = batch_size,
                          shuffle=True, collate_fn=collate_batch)

test_loader  = DataLoader(test, batch_size = batch_size,
                          shuffle=True, collate_fn=collate_batch)

## 6. Designing the model

In [ ]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, emb_dim, hid_dim, output_dim, num_layers, bidirectional, 
                 dropout):
        #input dim = how many vocab you have
        #emb dim = 300 --> we use fasttext
        #padding_idx tells this lookup table to ignore, and just randomize....
        #<unk>, <bos>, <eos>
        self.embedding_layer = nn.Embedding(input_dim, emb_dim, padding_idx=pad_ix)
        self.lstm            = nn.LSTM(emb_dim,
                                       hid_dim,
                                       num_layers = num_layers,
                                       bidrectional = bidirectional,
                                       dropout = dropout,  #dropout is applied between layers....
                                       batch_first=True)
        
        self.fc              = nn.Linear(hid_dim * 2, output_dim)
        
    def forward(self, x, lengths):
        #x: [batch size, seq len]
        
        embedded_x = self.embedding_layer(x)
        #x: [batch size, seq len, emb dim]
        
        #pack this embedded_x in such a way that RNN knows to ignore padding....
        #without batch_first = True; things will become [seq len, batch size, emb dim]
        pack_embedded = nn.utils.rnn.pack_padded_sequence(embedded_x, lengths.to('cpu'),
                                                          enforce_sorted=False,
                                                          batch_first = True
                                                          )
        
        #packed_outputs is basically all hidden states
        #h is the last hidden state
        #c is the last cell state
        packed_outputs, (h, _) = self.lstm(pack_embedded)
        
        #h: [num_layers * num_directions, batch_size, hidden dim]
        
        #it happens that because packed_outputs is all hidden states....some hidden states near the end is
        #hidden state for padding, pytorch guys help you
        #by using this pad_packed_sequence, then all the hidden states will only be not padding....
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first = True)
        #output: [batch size, seq len, direction * hidden sim]
        
        #last hidden state - concat last forward and backward states
        last_hidden_state = torch.cat((h[-1, :, :], h[-2, :, :]), dim = 1)
        #last_hidden_state: [batch_size, hidden_dim * 2]
        
        #for sentiment analysis.....what should i sent to my linear layer...
        return self.fc(last_hidden_state)  #[batch_size, output_dim]==> [batch_size, 4]

I try so many times and it doesn't work, it might be come from the libraries issues or my computer . I'm so sorry for unexpected problems .